# Проект "Музыка больших городов"

## Описание проекта

**Контекст исследования**:

Сравнение Москвы и Петербурга окружено мифами. Москва — мегаполис, подчинённый жёсткому ритму рабочей недели. Петербург — культурная столица, со своими вкусами. Используя данные Яндекс Музыки будет произведено сравненени и анализ поведения пользователей двух столиц.

**Цель исследования**: — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**:

Набор данных, `yandex_music_project.csv`, содержит информацию о поведении пользователей. Анализ проводится в соответствии со структурированным подходом:

1. **[Обзор данных](#1)**: первичное изучение набора данных.
2. **[Предобработка данных](#2)**: выявление и исправление критических ошибок в данных, чтобы обеспечить качество анализа.
3. **[Проверка гипотез](#3)**: тщательные тесты для подтверждения гипотез.
4. **[Заключение](#4)**: выводы по результатам исследования

**Описание столбцов**:
- `userID` — идентификатор пользователя;
- `Track` — название трека;
- `artist` — имя исполнителя;
- `genre` — название жанра;
- `City` — город пользователя;
- `time` — время начала прослушивания;
- `Day` — день недели.

## 1. Обзор данных <a id='1'> </a>

In [1]:
#import libraries
import pandas as pd

In [2]:
#import data
df = pd.read_csv('yandex_music_project.csv')

In [3]:
df.sample(5) #getting random rows of df

,userID,Track,artist,genre,City,time,Day
11996,D09603B7,Простреленный флаг,ДМЦ,rusrock,Moscow,20:06:44,Monday
59418,7CED98E9,NaN,NaN,jazz,Moscow,14:20:11,Wednesday
6047,43E93FD9,Carbon 12,Jlin,electronic,Moscow,08:49:07,Wednesday
24629,F50DF282,Mistake,Nickelback,rock,Moscow,13:13:06,Wednesday
36125,A1CD242C,Solaria,Solaris,electronic,Moscow,09:51:41,Friday


In [4]:
df.info() #general df info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


<div class="alert"; style="border-left: 7px solid pink">

**Мини-вывод**
- 65079 строк;
- В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку;
- Есть пропуски в стобцах `Track`, `artist` и `genre`;
- В названиях стобцов `userID`, `Track`, `City` и `Day` нарушен стиль.
- Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но так как встречаются пропуски в данных, а в названиях колонок есть расхождение с хорошим стилем, требуется устранить проблемы в данных, чтобы двигаться дальше.
</div>

## 2. Предобработка данных <a id='2'> </a>

### 2.1 Название столбцов

In [5]:
#column renaming
df = df.rename(columns = {'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})
df.head()

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


### 2.2 Пропуски

In [6]:
df.isna().sum() #counting missing values

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропуски в track и artist не важны и могут быть заменены явными обозначениями, но пропуски в genre могут затруднить сравнение музыкальных вкусов Москвы и Санкт-Петербурга, поэтому их также нужно заполнить и оценить влияние на расчёты.

In [7]:
df = df.fillna('unknown') #missing value processing
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### 2.3 Дубликаты

In [8]:
df.duplicated().sum() #counting duplicates

3826

In [9]:
df = df.drop_duplicates() #deletion of exact duplicates
df.duplicated().sum()

0

Избавимся от неявных дубликатов:

In [10]:
df['genre'].sort_values().unique() # getting unique genres

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [11]:
#processing implicit duplicates
df['genre'] = df['genre'].replace(['hip', 'hop', 'hip-hop'], 'hiphop')
df['genre'] = df['genre'].replace(['french', 'frankreich', 'französisch'], 'french')
df['genre'] = df['genre'].replace(['ruspop', 'russian'], 'ruspop')
df['genre'] = df['genre'].replace(['rockalternative', 'rockindie', 'rockother'], 'rock')
df['genre'] = df['genre'].replace(['rnb', 'rhythm'], 'rnb')
df['genre'] = df['genre'].replace(['metal', 'metalcore'], 'metal')
df['genre'] = df['genre'].replace(['triphop', 'trip'], 'triphop')
df['genre'] = df['genre'].replace(['newage', 'new'], 'newage')
df['genre'] = df['genre'].replace(['popdance', 'pop', 'popelectronic', 'popeurodance'], 'pop')
df['genre'] = df['genre'].replace(['country', 'american'], 'country')
df['genre'] = df['genre'].replace(['world', 'worldbeat'], 'world')
df['genre'] = df['genre'].replace(['electronic', 'electropop', 'электроника'], 'electronic')
df['genre'] = df['genre'].replace(['türk', 'türkçe'], 'turkish')

Выведем количество уникальных жанров после обработки:

In [12]:
len(df['genre'].sort_values().unique()) # getting unique genres after processing duplicates

270

Отметим также, что в данных есть жанр `ïîï`, который возможно является результатом неправильной кодировки и требуется дополнительной проверки.

<div class="alert"; style="border-left: 7px solid pink">

**Мини-вывод по предобработке**
- Название столбцов скорректированы;
- Пропуски обработаны (заменены на 'unknown');
- Явные и неявные дубликаты обработаны;
- Данные готовы для проверки гипотез.
</div>

## 3. Проверка гипотез <a id='3'> </a>

### 3.1 Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу (в данных есть информация о прослушиваниях только за эти дни).

Оценим активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе:

In [13]:
df.groupby('city')['track'].count() #сounting auditions in each city

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Далее сгруппируем данные по дню недели и посчитаем прослушивания в понедельник, среду и пятницу:

In [14]:
df.groupby('day')['track'].count() #сounting auditions in each city

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

Пользователи менее активны по средам, однакоо картина может измениться, если рассмотреть каждый город в отдельности.

Теперь напишем функцию `number_tracks`, которая посчитает прослушивания для заданного дня и города:

In [15]:
def number_tracks(day, city):
    track_list = df[df['city'] == city]
    track_list = track_list[track_list['day'] == day]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Создадим таблицу, показывающую прослушивания для каждого города в разрезе дней недели:

In [16]:
columns = ['сity'] + list(map(str.lower, df.day.unique()))
data = []

for city in df.city.unique():
    row = [city]
    for day in df.day.unique():
        num_tracks = number_tracks(day, city)
        row.append(num_tracks)
    data.append(row)

result_df = pd.DataFrame(data=data, columns=columns)

display(result_df)

,сity,wednesday,friday,monday
0,Saint-Petersburg,7003,5895,5614
1,Moscow,11056,15945,15740


Данные показывают разницу поведения пользователей. В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад. В Петербурге, наоборот, больше слушают музыку по средам. Следовательно, данные говорят в пользу первой гипотезы.

### 3.2 Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [17]:
#separating data
moscow_general = df[df['city'] == 'Moscow']
spb_general = df[df['city'] == 'Saint-Petersburg']

Далее создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm',
* последняя временная метка в формате 'hh:mm'.

Функция будет возвращать информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [18]:
# creating function
def genre_weekday(df, day, time1, time2):
    genre_df = df[df['day'] == day]
    genre_df = genre_df[(genre_df['time'] > time1) & (genre_df['time'] < time2)]
    genre_df = genre_df.groupby('genre')['genre'].count().sort_values(ascending = False)
    return genre_df[:10]

Cравним результаты функции для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [19]:
#moscow monday morning
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     483
rock           475
hiphop         286
ruspop         193
world          182
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [20]:
#spb monday morning
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          65
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [21]:
#moscow friday evening
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           518
dance          495
electronic     482
hiphop         273
world          208
ruspop         175
classical      163
alternative    163
rusrap         142
Name: genre, dtype: int64

In [22]:
#spb friday evening
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное различие — в московский рейтинг вошёл жанр `world`, а в петербургский — `джаз` и `классика`.
2. В Москве пропущенных значений оказалось так много, что значение `unknown` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### 3.1 Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве. А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу moscow_general по жанру и посчитайте прослушивания треков каждого жанра методом count(). Затем отсортируем результат в порядке убывания и сохраним его в таблице moscow_genres:

In [23]:
#moscow top genres
moscow_genres = moscow_general.groupby('genre')['genre'].count()
moscow_genres = moscow_genres.sort_values(ascending = False)
moscow_genres.head(10)

genre
pop            5893
dance          4435
rock           3969
electronic     3792
hiphop         2096
classical      1616
ruspop         1435
world          1433
alternative    1379
rusrap         1161
Name: genre, dtype: int64

Проделаем аналогичную работу для СПб:

In [24]:
#spb top genres
spb_genres = spb_general.groupby('genre')['genre'].count()
spb_genres = spb_genres.sort_values(ascending = False)
spb_genres.head(10)

genre
pop            2433
dance          1932
rock           1880
electronic     1738
hiphop          960
alternative     649
classical       646
ruspop          565
rusrap          564
world           516
Name: genre, dtype: int64

Гипотеза не подтвердилась:
- Поп-музыка — самый популярный жанр и в Москве, и в Спб.
- Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.

<div class="alert"; style="border-left: 7px solid pink">

**Мини-вывод**
- Первая гипотеза подтвердилась: в Москве пик прослушиваний приходится на понедельник и пятницу, а в СПб - наоборот;
- Вторая гипотеза подтвердлилась частично: в Москве и Петербурге слушают похожую музыку. Единственное различие — в московский рейтинг вошёл жанр `world`, а в петербургский — `джаз` и `классика`;
- Третья гипотеза не подтвердлилась: `поп-музыка` — самый популярный жанр и в Москве, и СПб, при этом `рэп` одинаково популярен в обоих городах.
</div>

## Заключение <a id='4'> </a>

В рамках исследования анализировалась информация о поведении слушателей Яндекс.Музыки в Москве и Санкт-Петербург.

**Предварительно данные были предобработаны**:
- Название столбцов скорректированы;
- Пропуски обработаны (заменены на 'unknown');
- Явные и неявные дубликаты обработаны.

**Далее было проверено три гипотезы и установлено**:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад. В Петербурге, наоборот, больше слушают музыку по средам. Следовательно, данные говорят в пользу первой гипотезы.

Таким образом, первая *гипотеза полностью подтвердилась*.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза *подтвердилась лишь отчасти*. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего, чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза *не подтвердилась*. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.